# Scrap Data From [ClinicalTrials](https://clinicaltrials.gov) & Load To MySQL DB
## [API](https://www.clinicaltrials.gov/api/gui/home) , [Data](https://clinicaltrials.gov/ct2/results)

## Library

In [1]:
# !pip install requests

In [2]:
import re
import json
import pymysql
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

print(f"Json Version:- {json.__version__}")        # 2.0.9
print(f"MySQL Version:- {pymysql.__version__}")    # 1.0.2
print(f"Request Version:- {requests.__version__}") # 2.26.0
print(f"Numpy Version:- {np.__version__}")         # 1.23.1
print(f"Pandas Version:- {pd.__version__}")        # 1.3.4

Json Version:- 2.0.9
MySQL Version:- 1.0.2
Request Version:- 2.26.0
Numpy Version:- 1.23.1
Pandas Version:- 1.3.4


## User Input

In [3]:
### Fields To Scrape
Field_List = ["NCTId", "BriefTitle", "Condition", "StudyType", "LeadSponsorName", "OrgFullName", "EnrollmentCount"]

SQL_CLM = "Row_No, NCT_Id, Study_Title, `Condition`, Study_Type, Sponsor_Name, OrgFullName, Number_Enrolled"

### API Prefix
API_Prefix = "https://www.clinicaltrials.gov/api/query/study_fields?"

Max_Scrape_Limit = 1000

### MySQL Credential
MySQL_Endpoint = "127.0.0.1"
MySQL_Port = "3306"
MySQL_UserName = "root"
MySQL_Password = "MySql@1234"
MySQL_DataBase = "clinical_studies"

Schema = "clinical_studies"
Table = "land_clinical_trials"

### DataBaseTable to Store Data

**CREATE SCHEMA** clinical_studies;

**USE** clinical_studies;

**CREATE TABLE IF NOT EXISTS** clinical_studies.land_clinical_trials(<br>
&emsp;&emsp;Row_No **BIGINT** , NCT_Id **VARCHAR**(100), <br>
&emsp;&emsp;Study_Title **VARCHAR**(12000), `Condition` **VARCHAR**(31000), <br>
&emsp;&emsp;Study_Type **VARCHAR**(700), Sponsor_Name **VARCHAR**(11000), <br>
&emsp;&emsp;OrgFullName **VARCHAR**(10700), Number_Enrolled **int** ) CHARACTER SET latin1 **;**

### Create MySQL Connection

In [4]:
MySQL_Connection = pymysql.connect(host= MySQL_Endpoint, 
                                   user= MySQL_UserName, 
                                   password= MySQL_Password, 
                                   db= MySQL_DataBase)

MySQL_Cursor = MySQL_Connection.cursor()

# Connection_STR = create_engine(f"mysql+pymysql://{MySQL_UserName}:{MySQL_Password}@{MySQL_Endpoint}/{MySQL_DataBase}")
# Connection    = Connection_STR.connect()

## UDF

In [5]:
def Create_Chunk_Range(Total_Row_No_ , Chunk_Size_):
    Chunk_Breakup_ = [i for i in range(0,Total_Row_No_,Chunk_Size_)]
    Chunk_Breakup_.append(Total_Row_No_)
    
    Chunk_Range_ = [(Chunk_Breakup_[i]+1 , "to" , Chunk_Breakup_[i+1]) for i in range(len(Chunk_Breakup_)-1)]
    
    return Chunk_Range_

In [6]:
def Data_Preparation(Record_):
    
    def Clean_Data(STR_):
        Clean_1 = re.sub(r'[^a-z A-Z |0|1|2|3|4|5|6|7|8|9| ,|?|$|.|!|:]',r'',STR_)
        return re.sub(r' +', ' ', Clean_1)
    
    Temp_Dict_ = {}
    Temp_Dict_.update({'Rank':int(Record_['Rank']),
                     'NCTId':str(Record_['NCTId'][0]),
                     'BriefTitle':Clean_Data(STR_ = str(", ".join(Record_['BriefTitle']))),
                     'Condition':Clean_Data(STR_ = str(", ".join(Record_['Condition']))),
                     'StudyType':Clean_Data(STR_ = str(", ".join(Record_['StudyType']))),
                     'LeadSponsorName':Clean_Data(STR_ = str(", ".join(Record_['LeadSponsorName']))),
                     'OrgFullName':Clean_Data(STR_ = str(", ".join(Record_['OrgFullName'])))  })

    if len(Record_['EnrollmentCount']) == 0:
        Temp_Dict_.update({'EnrollmentCount':0})
    else:
        Temp_Dict_.update({'EnrollmentCount':int(Record_['EnrollmentCount'][0])})
        
    return Temp_Dict_

## Scrape & Get Data

In [7]:
### Get Total Number Of Records
Res = requests.get(f"{API_Prefix}fmt=json")
Total_Records_Found = json.loads(Res.content.decode())['StudyFieldsResponse']['NStudiesFound']
print(f"Total Number Of Records - {Total_Records_Found}\n")

### Get Data Chunk Row Number, As Limitation For Each Chunk Is 1000
Chunk_Range = Create_Chunk_Range(Total_Row_No_ = Total_Records_Found, Chunk_Size_ = Max_Scrape_Limit)
print(f"### Printing Few Chunk Range. Total Chunk {len(Chunk_Range)}")
print(Chunk_Range[:5]," .....\n")

### Scrape & Get Data
Count = 0
Error_Dict = []
R_Field_List = ", ".join(Field_List)

for Each_Chunk_Range in Chunk_Range:
    Res = requests.get(f"{API_Prefix}fields={R_Field_List}&min_rnk={Each_Chunk_Range[0]}&max_rnk={Each_Chunk_Range[2]}&fmt=json")
    Data = json.loads(Res.content.decode())
    Data = Data['StudyFieldsResponse']['StudyFields']
    
    SQL_Val_List = []
    for Each_Record in Data:        
        Temp_Dict= Data_Preparation(Record_= Each_Record)        
        SQL_Val_List.append(str(tuple(Temp_Dict.values())))    
        del Temp_Dict
        
    try:
        SQL_Insert_Query = f"INSERT INTO {Schema}.{Table} ({SQL_CLM}) VALUES {', '.join(SQL_Val_List)}"
        MySQL_Cursor.execute(SQL_Insert_Query)
        MySQL_Connection.commit()
        
        Count += 1
        print(f"Successfully Load Chunk {Each_Chunk_Range}")
    except Exception as e:
        print(f"\n### Failed To Load Chunk {Each_Chunk_Range} ###")
        MySQL_Connection.rollback()
        Error_Dict.append(Each_Chunk_Range) 
        
    del SQL_Val_List
    del Data    
    
    ### Limit & Break
    if Count == 20:
        break
        
print(f"\nAll Done !!! \nTotal Chunk Load {Count}")

Total Number Of Records - 425817

### Printing Few Chunk Range. Total Chunk 426
[(1, 'to', 1000), (1001, 'to', 2000), (2001, 'to', 3000), (3001, 'to', 4000), (4001, 'to', 5000)]  .....

Successfully Load Chunk (1, 'to', 1000)
Successfully Load Chunk (1001, 'to', 2000)
Successfully Load Chunk (2001, 'to', 3000)
Successfully Load Chunk (3001, 'to', 4000)
Successfully Load Chunk (4001, 'to', 5000)
Successfully Load Chunk (5001, 'to', 6000)
Successfully Load Chunk (6001, 'to', 7000)
Successfully Load Chunk (7001, 'to', 8000)
Successfully Load Chunk (8001, 'to', 9000)
Successfully Load Chunk (9001, 'to', 10000)
Successfully Load Chunk (10001, 'to', 11000)
Successfully Load Chunk (11001, 'to', 12000)
Successfully Load Chunk (12001, 'to', 13000)
Successfully Load Chunk (13001, 'to', 14000)
Successfully Load Chunk (14001, 'to', 15000)
Successfully Load Chunk (15001, 'to', 16000)
Successfully Load Chunk (16001, 'to', 17000)
Successfully Load Chunk (17001, 'to', 18000)
Successfully Load Chunk (1

### View Data

In [8]:
MySQL_Cursor.execute(f"SELECT COUNT(*) from {Schema}.{Table}")
Total_Records_In_DB = MySQL_Cursor.fetchall()[0][0]
print(f"Total Record in MySQL DB - {Total_Records_In_DB}\n")

MySQL_Cursor.execute(f"SELECT * from {Schema}.{Table} LIMIT 5")
MySQL_TBL_Rows = MySQL_Cursor.fetchall()
Scraped_Data = pd.DataFrame(MySQL_TBL_Rows, columns= SQL_CLM.split(","))
Scraped_Data

Total Record in MySQL DB - 20000



,Row_No,NCT_Id,Study_Title,`Condition`,Study_Type,Sponsor_Name,OrgFullName,Number_Enrolled
0,1,NCT05516706,Comparison of Dynamic Stretching and Plyometri...,Sports Physical Therapy,Interventional,Riphah International University,Riphah International University,22
1,2,NCT05516693,"Chewing, Swallowing and Orofacial Motricity in...","Mastication Disorder, Swallowing Disorder, Oro...",Interventional,Centro Universitrio de Anapolis,Centro Universitrio de Anapolis,80
2,3,NCT05516680,Effects and Central Mechanism of Electroacupun...,"Poststroke Depression, Healthy",Interventional,The Affiliated Nanjing Drum Tower Hospital of ...,The Affiliated Nanjing Drum Tower Hospital of ...,64
3,4,NCT05516667,Pilot Prospective Cohort of Patients With Alzh...,Alzheimer Disease,Observational,"University Hospital, Toulouse","University Hospital, Toulouse",750
4,5,NCT05516654,Type VI Secretion System in Klebsiella Pneumoniae,Isolation and Identification of K. Pneumoniae ...,Observational,Sohag University,Sohag University,70


### Close MySQL DB Connection

In [9]:
MySQL_Cursor.close()
MySQL_Connection.close()